In [1]:
from flurs.datasets import fetch_movielens
from flurs.recommender import BRISMFRecommender, MFRecommender
from flurs.forgetting import ForgetUnpopularItems, NoForgetting, UserFactorFading, SDUserFactorFading
from flurs.evaluator import Evaluator

import logging
import os
import sys

ML_PATH = 'D:\\recsys\\datasets\\movielens\\ml-1m\\'

program = os.path.basename(ML_PATH)
logger = logging.getLogger(program)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info('running %s' % ' '.join(sys.argv))

logging.info('converting data into FluRS input object')
data = fetch_movielens(data_home=ML_PATH, size='1m')

2019-03-10 17:57:08,769 : INFO : running D:\Anaconda3\envs\recsys\lib\site-packages\ipykernel_launcher.py -f C:\Users\Eduardo Ferreira\AppData\Roaming\jupyter\runtime\kernel-9732bd02-f2d1-4985-85e0-a48d8ca29b74.json
2019-03-10 17:57:08,769 : INFO : converting data into FluRS input object


In [2]:
recommender_list = [MFRecommender]
forgetting_list = [SDUserFactorFading(alpha=1.000001), UserFactorFading(fade=.2), NoForgetting(), ForgetUnpopularItems()]
experiments = []
for Recommender in recommender_list:
    for Forgetting in forgetting_list:
        logging.info('initialize recommendation model {} with {}'.format(Recommender.__name__, Forgetting.__class__.__name__))
        rec = Recommender(forgetting = Forgetting)
        rec.initialize()
        evaluator = Evaluator(rec, data.can_repeat)

        n_batch_train = int(data.n_sample * 0.2)  # 20% for pre-training to avoid cold-start
        n_batch_test = int(data.n_sample * 0.1)  # 10% for evaluation of pre-training
        batch_tail = n_batch_train + n_batch_test

        # pre-train
        # 20% for batch training | 10% for batch evaluate
        # after the batch training, 10% samples are used for incremental updating
        logging.info('batch pre-training before streaming input')
        evaluator.fit(
            data.samples[:n_batch_train],
            data.samples[n_batch_train:batch_tail],
            n_epoch=1  # single pass even for batch training
        )
        
        # 70% incremental evaluation and updating
        logging.info('incrementally predict, evaluate and update the recommender')
        experiments.append((evaluator.evaluate(data.samples[batch_tail:]), Recommender, Forgetting))

2019-03-10 17:57:12,967 : INFO : initialize recommendation model MFRecommender with SDUserFactorFading
2019-03-10 17:57:12,968 : INFO : batch pre-training before streaming input
2019-03-10 17:57:27,968 : INFO : incrementally predict, evaluate and update the recommender
2019-03-10 17:57:27,970 : INFO : initialize recommendation model MFRecommender with UserFactorFading
2019-03-10 17:57:27,970 : INFO : batch pre-training before streaming input
2019-03-10 17:57:40,820 : INFO : incrementally predict, evaluate and update the recommender
2019-03-10 17:57:40,820 : INFO : initialize recommendation model MFRecommender with NoForgetting
2019-03-10 17:57:40,820 : INFO : batch pre-training before streaming input
2019-03-10 17:57:53,640 : INFO : incrementally predict, evaluate and update the recommender
2019-03-10 17:57:53,648 : INFO : initialize recommendation model MFRecommender with ForgetUnpopularItems
2019-03-10 17:57:53,648 : INFO : batch pre-training before streaming input
2019-03-10 17:58:0

In [3]:
for evaluator, recommender, forgetting in experiments:
    filename = "D:\\recsys\\flurs\\results\\{}_{}.dat".format(recommender.__name__, forgetting.__class__.__name__)
    print("Abrindo arquivo {} ...".format(filename))
    with open(filename, 'w+') as f:
        print("Começando geração de resultados ...")
        for instance in evaluator:
            f.write(str(instance))
    print("Arquivo {} completo.".format(filename))
    

Abrindo arquivo D:\recsys\flurs\results\MFRecommender_SDUserFactorFading.dat ...
Começando geração de resultados ...
Arquivo D:\recsys\flurs\results\MFRecommender_SDUserFactorFading.dat completo.
Abrindo arquivo D:\recsys\flurs\results\MFRecommender_UserFactorFading.dat ...
Começando geração de resultados ...
Arquivo D:\recsys\flurs\results\MFRecommender_UserFactorFading.dat completo.
Abrindo arquivo D:\recsys\flurs\results\MFRecommender_NoForgetting.dat ...
Começando geração de resultados ...
Arquivo D:\recsys\flurs\results\MFRecommender_NoForgetting.dat completo.
Abrindo arquivo D:\recsys\flurs\results\MFRecommender_ForgetUnpopularItems.dat ...
Começando geração de resultados ...
Arquivo D:\recsys\flurs\results\MFRecommender_ForgetUnpopularItems.dat completo.
